In [2]:
import mlflow
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout

/home/vikoluna/NeuralCatDog/venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/vikoluna/NeuralCatDog/venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2023-10-01 15:47:42.167372: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 15:47:42.253549: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-01 15:47:42.253610: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register c

In [3]:
# Importamos los datos de entrenamiento y prueba de nuestra red
train_dir = '/home/vikoluna/NeuralCatDog/data/train'
test_dir = '/home/vikoluna/NeuralCatDog/data/test'

# Esto es por perros y gatos (0 y 1 o sea seran 2 clases)
num_class = 2
# Es el número de épocas
epoch = 10

# Numero de imagenes
num_train = 10001
num_test = 2501

# Como tenemos imagenes de distintos tamaños, vamos a ajustar las imagenes a nuevas dimensiones
ih, iw = 150, 150
# Definimos la forma, alto, ancho y canales RGB
input_shape = (ih, iw, 3)

# Definimos el tamaño del batch
batch_size = 20

epoch_steps = num_train // batch_size
test_steps = num_test // batch_size

In [4]:
# Indica que reescale cada canal con valor entre 0 y 1
gentrain = ImageDataGenerator(rescale=1. / 255.)

train = gentrain.flow_from_directory(train_dir,
                                     batch_size=batch_size,
                                     target_size=(iw, ih),
                                     class_mode='binary')

gentest = ImageDataGenerator(rescale=1. / 255.)

test = gentest.flow_from_directory(train_dir,
                                     batch_size=batch_size,
                                     target_size=(iw, ih),
                                     class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [5]:
# Comenzamos a crear nuestra arquitectura de red secuencial
model = Sequential()

# Comenzaremos agregando una capa de entrada
model.add(Conv2D(10, (5, 5), input_shape=(ih, iw, 3)))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Capas ocultas
model.add(Conv2D(10, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(20, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Capa de salida
# Aplanamos la imagen
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('softmax'))
# Regularización
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compilamos el modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


with mlflow.start_run() as run:
    model.fit_generator(
        train,
        steps_per_epoch=epoch_steps,
        epochs=epoch,
        validation_data=test,
        validation_steps=test_steps
    )

# Salvamos el modelo
# model.save()

2023-10-01 15:48:09.111618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 15:48:09.757450: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10


/tmp/ipykernel_8852/318020962.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
2023-10-01 15:48:11.675063: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25482600 exceeds 10% of free system memory.
2023-10-01 15:48:11.675966: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25482600 exceeds 10% of free system memory.
2023-10-01 15:48:11.691045: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 29036160 exceeds 10% of free system memory.
2023-10-01 15:48:11.784865: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31974000 exceeds 10% of free system memory.


  2/500 [..............................] - ETA: 2:20 - loss: 0.6971 - accuracy: 0.4750 

2023-10-01 15:48:11.966106: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25482600 exceeds 10% of free system memory.


500/500 [==============================] - 145s 288ms/step - loss: 0.6972 - accuracy: 0.5025 - val_loss: 0.6930 - val_accuracy: 0.5096
Epoch 2/10
500/500 [==============================] - 118s 237ms/step - loss: 0.6939 - accuracy: 0.4982 - val_loss: 0.6932 - val_accuracy: 0.4964
Epoch 3/10
500/500 [==============================] - 119s 238ms/step - loss: 0.6936 - accuracy: 0.5011 - val_loss: 0.6932 - val_accuracy: 0.4872
Epoch 4/10
500/500 [==============================] - 125s 249ms/step - loss: 0.6934 - accuracy: 0.5026 - val_loss: 0.6932 - val_accuracy: 0.4968
Epoch 5/10
500/500 [==============================] - 117s 235ms/step - loss: 0.6933 - accuracy: 0.5032 - val_loss: 0.6932 - val_accuracy: 0.4924
Epoch 6/10
500/500 [==============================] - 118s 236ms/step - loss: 0.6933 - accuracy: 0.4973 - val_loss: 0.6932 - val_accuracy: 0.4984
Epoch 7/10
500/500 [==============================] - 117s 233ms/step - loss: 0.6933 - accuracy: 0.5005 - val_loss: 0.6931 - val_accura

Traceback (most recent call last):
  File "/home/vikoluna/NeuralCatDog/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_8852/318020962.py", line 44, in <module>
    model.save()
  File "/home/vikoluna/NeuralCatDog/venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/vikoluna/NeuralCatDog/venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 67, in error_handler
    filtered_tb = _process_traceback_frames(e.__traceback__)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Model.save() missing 1 required positional argument: 'filepath'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/vikoluna/NeuralCatDog/venv/lib/python3.11/site-packages/IPython/core/i

In [1]:
from PIL import Image

# Abre la imagen
imagen = Image.open('/home/vikoluna/NeuralCatDog/data/test/cat/cat.10205.jpg')  # Reemplaza 'imagen.jpg' con la ruta de tu imagen

# Obtiene el tamaño de la imagen (ancho x alto)
ancho, alto = imagen.size

# Muestra el tamaño de la imagen
print(f"Ancho: {ancho}px")
print(f"Alto: {alto}px")

Ancho: 500px
Alto: 374px
